# Week 6: Merging data with Pandas, Part 1
What's the relationship between vaccination rates and votes in the recall election [using the `merge()` method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)

#### Load our Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

---

### Vax percentage

#### LA Times [coronavirus repo](https://github.com/datadesk/california-coronavirus-data)

In [4]:
vax_url = "https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/cdph-vaccination-county-totals.csv"

#### Read weekly vaccination totals by county data

In [5]:
vax_df = pd.read_csv(vax_url, dtype={"fips": str})

#### How many weeks is this (don't forget to sort)? 

In [6]:
vax_df.sort_values("date", ascending=False).head(5)

,date,county,fips,population,doses_administered,new_doses_administered,pfizer_doses,new_pfizer_doses,moderna_doses,new_moderna_doses,jj_doses,new_jj_doses,partially_vaccinated,new_partially_vaccinated,at_least_one_dose,new_at_least_one_dose,fully_vaccinated,new_fully_vaccinated,partially_vaccinated_percent,at_least_one_dose_percent,fully_vaccinated_percent
0,2022-02-15,Alameda,001,1643700.0,3485200,2534,2127016.0,1470,1060982.0,606,133395.0,30,108068.0,395,1447101.0,402,1339033.0,552,0.065747,0.880392,0.814646
45,2022-02-15,Sierra,091,2930.0,3929,9,567.0,1,3232.0,1,74.0,0,116.0,5,1675.0,5,1559.0,4,0.039590,0.571672,0.532082
32,2022-02-15,Riverside,065,2383286.0,3547225,1911,2058984.0,965,1300748.0,525,96269.0,28,166410.0,359,1584812.0,374,1418402.0,567,0.069824,0.664969,0.595146
33,2022-02-15,Sacramento,067,1510023.0,2602807,1630,1452709.0,835,988452.0,453,67939.0,43,92216.0,230,1112681.0,243,1020465.0,455,0.061069,0.736864,0.675794
34,2022-02-15,San Benito,069,59416.0,110868,71,60297.0,33,42528.0,25,3916.0,0,4502.0,11,48622.0,11,44120.0,18,0.075771,0.818332,0.742561


In [7]:
vax_df["date"].max()

'2022-02-15'

In [8]:
vax_df["date"].min()

'2020-01-05'

#### Select the most recent week

In [9]:
most_recent = vax_df[vax_df["date"] == vax_df["date"].max()]

#### Get only the columns we need

In [10]:
vax_slim = most_recent[["county", "fips", "fully_vaccinated_percent", "population"]]

In [11]:
vax_slim.head()

,county,fips,fully_vaccinated_percent,population
0,Alameda,001,0.814646,1643700.0
1,Alpine,003,0.625436,1148.0
2,Amador,005,0.542652,37829.0
3,Butte,007,0.502600,227075.0
4,Calaveras,009,0.532287,45235.0


---

### Recall vote

#### Read the county-level election results

In [12]:
recall_df = pd.read_json(
    "../../data/raw/gov_race_change_counties.json", dtype={"fips": str}
)

#### Rename columns

In [13]:
recall_df.rename(
    columns={
        "d_pct_21": "no_percent",
        "r_pct_21": "yes_percent",
        "leader_2021": "winner_2021",
    },
    inplace=True,
)

In [14]:
# This will also work without the inplace argument
# recall_df = recall_df.rename(columns={"d_pct_21": "no_percent"})

#### What's that look like? 

In [15]:
recall_df.head()

,county,fips,dem_2018,dem_2021,rep_2018,rep_2021,votes_2018,votes_2021,d_pct_18,no_percent,r_pct_18,yes_percent,d_change,r_change,winner_2021,leader_2018
0,Alameda,001,462558,287014,111677,60716,574235,347730,80.6,82.5,19.4,17.5,1.9,-1.9,D,D
1,Alpine,003,386,340,229,218,615,558,62.8,60.9,37.2,39.1,-1.9,1.9,D,D
2,Amador,005,6237,6411,11356,10971,17593,17382,35.5,36.9,64.5,63.1,1.4,-1.4,R,R
3,Butte,007,41500,31774,47226,35182,88726,66956,46.8,47.5,53.2,52.5,0.7,-0.7,R,R
4,Calaveras,009,7765,7106,13845,11450,21610,18556,35.9,38.3,64.1,61.7,2.4,-2.4,R,R


#### Which county was most against the recall?

In [16]:
recall_df[recall_df["no_percent"] == recall_df["no_percent"].max()]

,county,fips,dem_2018,dem_2021,rep_2018,rep_2021,votes_2018,votes_2021,d_pct_18,no_percent,r_pct_18,yes_percent,d_change,r_change,winner_2021,leader_2018
37,San Francisco,075,312181,231861,49181,35571,361362,267432,86.4,86.7,13.6,13.3,0.3,-0.3,D,D


#### For it? 

In [17]:
recall_df[recall_df["yes_percent"] == recall_df["yes_percent"].max()]

,county,fips,dem_2018,dem_2021,rep_2018,rep_2021,votes_2018,votes_2021,d_pct_18,no_percent,r_pct_18,yes_percent,d_change,r_change,winner_2021,leader_2018
17,Lassen,035,2043,1357,6973,6590,9016,7947,22.7,17.1,77.3,82.9,-5.6,5.6,R,R


#### Get only the columns we need

In [18]:
recall_df_slim = recall_df[
    ["county", "fips", "no_percent", "yes_percent", "winner_2021"]
]

In [19]:
recall_df_slim.head()

,county,fips,no_percent,yes_percent,winner_2021
0,Alameda,001,82.5,17.5,D
1,Alpine,003,60.9,39.1,D
2,Amador,005,36.9,63.1,R
3,Butte,007,47.5,52.5,R
4,Calaveras,009,38.3,61.7,R


---

## Merge the vax data with the election results

#### Use the `merge()` method

In [20]:
merged = pd.merge(recall_df_slim, vax_slim, on=["fips", "county"])

In [21]:
merged.corr(method="pearson")

,no_percent,yes_percent,fully_vaccinated_percent,population
no_percent,1.000000,-1.000000,0.877600,0.275861
yes_percent,-1.000000,1.000000,-0.877600,-0.275861
fully_vaccinated_percent,0.877600,-0.877600,1.000000,0.295092
population,0.275861,-0.275861,0.295092,1.000000


---

In [22]:
domain = ["R", "D"]
range_ = ["#d94f54", "#5789b8"]

alt.Chart(merged).mark_circle(size=100).encode(
    x=alt.X(
        "no_percent",
        scale=alt.Scale(domain=[10, 100]),
        axis=alt.Axis(tickCount=6, title="No percentage"),
    ),
    y=alt.Y(
        "fully_vaccinated_percent",
        scale=alt.Scale(domain=[0.2, 1]),
        axis=alt.Axis(tickCount=6, title="One dose percentage", format="%"),
    ),
    #     size="population",
    color=alt.Color("winner_2021", scale=alt.Scale(domain=domain, range=range_)),
    #     tooltip=["county", "population"],
).properties(width=500, height=500)

alt.Chart(...)

In [23]:
merged["fully_vaccinated_percent"] = round(merged["fully_vaccinated_percent"] * 100, 2)

In [24]:
merged.to_csv("../../data/processed/no_vax_corr.csv", index=False)

---